# Generating Text by Masking

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install transformers

     |████████████████████████████████| 2.1MB 5.5MB/s 
     |████████████████████████████████| 3.3MB 19.3MB/s 
     |████████████████████████████████| 901kB 45.2MB/s 


### Loading our pretrained models and HuBERT tokenizer

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
#from torch.nn import functional as F
#import torch

# #a szavak tokenizálója
tokenizer = BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc")
#model = BertForMaskedLM.from_pretrained("SZTAKI-HLT/hubert-base-cc") #comment out if you are using your model!!

In [ ]:
#PATH = "betanitas 8 25 epoch 64 batch mondat"

from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/content/drive/MyDrive/mese_modellek/betanítás 2 15 epoch 128 batch paragraph",
    #model=BertForMaskedLM.from_pretrained(PATH,local_files_only=True), #load your model!!!
    tokenizer=BertTokenizer.from_pretrained("SZTAKI-HLT/hubert-base-cc",local_files_only=False)
)

In [ ]:
# route to the generating folder with the text files
genetaringfolderpath='/content/drive/MyDrive/PyNLP/'

In [ ]:
#loading the input text
with open(genetaringfolderpath+'piroska_es_a_farkas.txt') as file_in:
    text = file_in.read()

In [ ]:
#sliding window file in case colab crashes while we generate text
with open(genetaringfolderpath+'csuszoablak_counter.txt') as file_in_k:
    k = file_in_k.read()
k=int(k)
print(k)

11


In [ ]:
text

'PIROSKA ÉS A FARKAS\nVolt egyszer egy kedves, aranyos kislány; aki csak ismerte, mindenki kedvelte, de legjobban\nmégis a nagymamája szerette: a világ minden kincsét neki adta volna. Egyszer vett neki egy\npiros bársonysapkát. A kislánynak annyira tetszett a sapka, hogy mindig csak ezt hordta; el is\nnevezték róla Piroskának.\nPiroskáék bent laktak a faluban, nagymama pedig kint az erdőben, egy takaros kis házban.\nEgy szép napon azt mondja Piroskának az édesanyja:\n- Gyere csak, kislányom! Itt van egy kalács meg egy üveg bor, vidd el a nagymamának. Beteg\nis, gyönge is szegényke, jól fog esni neki. Indulj szaporán, mielőtt beáll a hőség. Aztán\nszépen, rendesen menj, ne szaladgálj le az útról, mert elesel, és összetörik az üveg, kifolyik a\nbor, és akkor mit iszik a nagymama! Ha pedig odaérsz, ne bámészkodj összevissza a szobában; az legyen az első dolgod, hogy illedelmesen jó reggelt kívánj.\n- Bízzad csak rám, édesanyám, minden úgy lesz, ahogy mondod - felelte Piroska az intelemre,

Parameters

In [ ]:
random_pred_parameter=20

# tokens or words we don't want in our generated sentences
excluded_words = ['[ P A D ]', '[ U N K ]', ';', ':', '?', '!', ',', '"', '$', '[', ']', '%', '&', '(', ')', '*', '+', '-', '/', '0',  '1', '2', '3', '4', '5', '6', '7', '8', '9', '<', '=', '>', '<', '”', '«', '»', '–', '“', '„', ] 

In [ ]:
len(excluded_words)

39

### First idea

Takes the input text and randomly masks and replaces tokens n_iter times.

In [ ]:
import random

In [ ]:
def modify_text(sentence, exc_words, n_iter, prob=0, random_pred_parameter=20):
  #sentence: the text to modify, generate from
  #exc_words: words, tokens we want to avoid when generating the new texts
  #n_iter: how many times we want to swap tokens
  #prob: in each iteration we add a mask token at the end of the sentence with this probability
  #all_sentences = [sentence] #saving all the generated sentences in the process

  while n_iter > 0:

    sentence_words = sentence.split()
    sentence_words[random.choice(range(len(sentence_words)))] = tokenizer.mask_token
    masked_sentence = " ".join(sentence_words)
    chosen_sentence = fill_mask(masked_sentence, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]

    while bool([exclword for exclword in excluded_words if(exclword in chosen_sentence['token_str'])]):
      chosen_sentence = fill_mask(masked_sentence, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]
    sentence = chosen_sentence['sequence']
    k = random.random()
    if k < prob:
      sentence = sentence + ' ' + tokenizer.mask_token
      chosen_sentence = fill_mask(sentence, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]

      while bool([exclword for exclword in excluded_words if(exclword in chosen_sentence['token_str'])]):
        chosen_sentence = fill_mask(sentence, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]
      sentence = chosen_sentence['sequence']
    #all_sentences.append(sentence) # not yet returned by the function
    n_iter -= 1
  return str(sentence)

We used 128 token batches for generating since the masking pipeline can only take 512 tokens maximum but also if the batches are too large the process takes too long.

In [ ]:
n_iter=(128/2) #azaz az ablakon belül átag minden 2. szót letakarjuk random módon persze

In [ ]:
input_len = len(text.split())
k=0
while k<input_len/128:
    text_2 = text.split()[k*128:(k+1)*128] #egyszerre mindig csak egy 64 szavas részleten futtatjuk a függvényt, majd ezt hozzáírjuk a kimenethez
    text_2=" ".join(text_2)
    generated = modify_text(text_2, excluded_words, n_iter,random_pred_parameter=20)
    with open(genetaringfolderpath+'randomizalt_kimenet_15128parag.txt', 'a') as file_out:
        file_out.write(generated)
        file_out.write('.')
    k+=1
    with open(genetaringfolderpath+'csuszoablak_counter.txt', 'w') as file_out_k:
        file_out_k.write(str(k))
    print("progress :")
    print(k/(input_len/128))

progress :
0.09162491052254831
progress :
0.18324982104509663
progress :
0.27487473156764497
progress :
0.36649964209019326
progress :
0.4581245526127416
progress :
0.5497494631352899
progress :
0.6413743736578382
progress :
0.7329992841803865
progress :
0.8246241947029349
progress :
0.9162491052254832
progress :
1.0078740157480315


### Prime method

Takes tokens on prime indices (2,3,5 in our case) and does the mask and replace procedure.

In [ ]:
def prime(i, primes):
    for prime in primes:
        if not (i == prime or i % prime):
            return False
    primes.add(i)
    return i

def historic(n): # get first n primes
    primes = set([2])
    i, p = 2, 0
    while True:
        if prime(i, primes):
            p += 1
            if p == n:
                return primes
        i += 1

In [ ]:
def modify_text_prime(text, exc_words, n_prime,random_pred_parameter=20):
  #text: the text to modify, generate from
  #exc_words: words, tokens we want to avoid when generating the new texts
  #n_prime: how many primes we want
  text_words = text.split()
  for i in historic(n_prime):
    for j in range(len(text_words)):
      if j%i==0:
        text_words = text.split()
        try:
          text_words[j-1] = tokenizer.mask_token
        except IndexError:
          continue
        masked_text = " ".join(text_words)
        chosen_text = fill_mask(masked_text, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]

        while bool([exclword for exclword in excluded_words if(exclword in chosen_text['token_str'])]):
          chosen_text = fill_mask(masked_text, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]
        text = chosen_text['sequence']

  return str(text)

In [ ]:
n_prime=3

In [ ]:
input_len = len(text.split())
k=0
while k<input_len/128:
    text_2 = text.split()[k*128:(k+1)*128]
    text_2=" ".join(text_2)
    generated = modify_text_prime(text_2, excluded_words,n_prime)
    with open(genetaringfolderpath+'randomizalt_kimenet_prim_15128parag.txt', 'a') as file_out:
        file_out.write(generated)
        file_out.write('.')
    k+=1
    with open(genetaringfolderpath+'csuszoablak_counter.txt', 'w') as file_out_k:
        file_out_k.write(str(k))
    print("progress :")
    print(k/(input_len/128))

progress :
0.09162491052254831
progress :
0.18324982104509663
progress :
0.27487473156764497
progress :
0.36649964209019326
progress :
0.4581245526127416
progress :
0.5497494631352899
progress :
0.6413743736578382
progress :
0.7329992841803865
progress :
0.8246241947029349
progress :
0.9162491052254832
progress :
1.0078740157480315


### Context idea

Finally here we take 5 sentences at a time and only mask and replace on the final one (the other 4 are there for context). This method tried to resemble the original left to right text generating with little success.

In [ ]:
import re
import nltk
nltk.download('punkt')
from nltk import tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
text_test = "Aztán egyszer csak jött ám a sárkány tüzet okádva. A fiú gyorsan megsuhintotta a kardot, és mind a hét fejét levágta egyetlen csapással. Megörült erre a leány, nyomban asztalt terített, és kérte a fiút, hogy maradjon nála. A királyfi azonban azt felelte, hogy erről szó sem lehet, mert neki előbb meg kell keresnie az aranymadarat. Megkérdezte a leányt, nem tudná-e megmondani, merre találhatna rá?"

In [ ]:
text_test

'Aztán egyszer csak jött ám a sárkány tüzet okádva. A fiú gyorsan megsuhintotta a kardot, és mind a hét fejét levágta egyetlen csapással. Megörült erre a leány, nyomban asztalt terített, és kérte a fiút, hogy maradjon nála. A királyfi azonban azt felelte, hogy erről szó sem lehet, mert neki előbb meg kell keresnie az aranymadarat. Megkérdezte a leányt, nem tudná-e megmondani, merre találhatna rá?'

In [ ]:
def context_gen_2(text, excluded_words,random_pred_parameter=20):
  sentences = re.findall('.*?[.!\?]', text_test)
  sent_ns = []
  sent_lengths = []
  for i in range(2,len(sentences)):
    sent_ns.append(sentences[i-2]+sentences[i-1]+ sentences[i])
  for i in range(len(sentences)):
    sent_lengths.append(len(sentences[i].split()))
  output = sentences[0]+sentences[1]
  for i in range(len(sent_ns)):
    sent_n_split = sent_ns[i].split()
    c=0
    while c<3*sent_lengths[i+2]:
      if c > 0:
        sent_n_split=new_text.split()
      try:
        sent_n_split[random.choice(range(sum(sent_lengths[i:i+2]),sum(sent_lengths[i:i+3])))] = tokenizer.mask_token
      except IndexError:
        c += 1
        continue
      masked_text = " ".join(sent_n_split)
    
      chosen_text = fill_mask(masked_text, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]

      while bool([exclword for exclword in excluded_words if(exclword in chosen_text['token_str'])]):
        chosen_text = fill_mask(masked_text, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]
        new_text = chosen_text['sequence']
      new_text = chosen_text['sequence']
      text_split = new_text.split()
      c+=1
      #print('done a masking iter')
    last_sent = text_split[(sent_lengths[i]+sent_lengths[i+1]):]  
    output += " ".join(last_sent)+'.'
  return output

In [ ]:
context_gen_2(text_test,excluded_words)

'Aztán egyszer csak jött ám a sárkány tüzet okádva. A fiú gyorsan megsuhintotta a kardot, és mind a hét fejét levágta egyetlen csapással.Megörültek a barátok a beszéd a..tizenkét gyermek azonban erősen nézte azért a hanem amikor hogy kit merítsen ű.nem keresem bizony én azt én mire találhatna az.'

In [ ]:
text = text.replace('\n', ' ')

In [ ]:
def context_gen_4(text, excluded_words,random_pred_parameter=20):
  sentences = tokenize.sent_tokenize(text)
  sent_ns = []
  sent_lengths = []
  for i in range(4,len(sentences)):
    sent_ns.append(sentences[i-4]+sentences[i-3]+sentences[i-2]+sentences[i-1]+ sentences[i])
  for i in range(len(sentences)):
    sent_lengths.append(len(sentences[i].split()))
  output = sentences[0]+sentences[1]+sentences[2]+sentences[3]
  for i in range(len(sent_ns)):
    sent_n_split = sent_ns[i].split()
    c=0
    l = 0
    while c<3*sent_lengths[i+4]:
      if l > 0:
        sent_n_split=new_text.split()
      try:
        sent_n_split[random.choice(range(sum(sent_lengths[i:i+4]),sum(sent_lengths[i:i+5])))] = tokenizer.mask_token
      except IndexError:
        c += 1
        text_split=sent_n_split
        continue
      masked_text = " ".join(sent_n_split)
    
      chosen_text = fill_mask(masked_text, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]

      while bool([exclword for exclword in excluded_words if(exclword in chosen_text['token_str'])]):
        chosen_text = fill_mask(masked_text, top_k=random_pred_parameter)[random.choice(range(random_pred_parameter))]
        new_text = chosen_text['sequence']
      new_text = chosen_text['sequence']
      text_split = new_text.split()
      c+=1
      l+=1
      #print('done a masking iter')
    last_sent = text_split[(sent_lengths[i]+sent_lengths[i+1]+sent_lengths[i+2]+sent_lengths[i+3]):]  
    print(last_sent)
    output += " ".join(last_sent)+'.'
    print(output)
  return output

In [ ]:
context_gen_4(text,excluded_words)

['takaros', 'kertben', 'erdei', 'csak', 'a', 'nagy', 'szólt', 'nekem', 'a', 'versetj', 'szép', 'kislányom!']
PIROSKA ÉS A FARKAS Volt egyszer egy kedves, aranyos kislány; aki csak ismerte, mindenki kedvelte, de legjobban mégis a nagymamája szerette: a világ minden kincsét neki adta volna.Egyszer vett neki egy piros bársonysapkát.A kislánynak annyira tetszett a sapka, hogy mindig csak ezt hordta; el is nevezték róla Piroskának.Piroskáék bent laktak a faluban, nagymama pedig kint az erdőben, egy takaros kis házban.takaros kertben erdei csak a nagy szólt nekem a versetj szép kislányom!.
['már', 'csak', 'Itt', 'itt', 'egy', 'leány', 'nem', 'akar', 'mondani', 'valamit', 'erről', 'világnak']
PIROSKA ÉS A FARKAS Volt egyszer egy kedves, aranyos kislány; aki csak ismerte, mindenki kedvelte, de legjobban mégis a nagymamája szerette: a világ minden kincsét neki adta volna.Egyszer vett neki egy piros bársonysapkát.A kislánynak annyira tetszett a sapka, hogy mindig csak ezt hordta; el is nevezték 

"PIROSKA ÉS A FARKAS Volt egyszer egy kedves, aranyos kislány; aki csak ismerte, mindenki kedvelte, de legjobban mégis a nagymamája szerette: a világ minden kincsét neki adta volna.Egyszer vett neki egy piros bársonysapkát.A kislánynak annyira tetszett a sapka, hogy mindig csak ezt hordta; el is nevezték róla Piroskának.Piroskáék bent laktak a faluban, nagymama pedig kint az erdőben, egy takaros kis házban.takaros kertben erdei csak a nagy szólt nekem a versetj szép kislányom!.már csak Itt itt egy leány nem akar mondani valamit erről világnak.aztán ha holnapután épp hogy elmegynek.Induljunk ki elkészül minden..hőség.. jó halkanon jön még az ugrás s most mindegyek mert talán meghal nagymama!.mert ezek tegnap nem mennek nagy házban kint vana pap is van éjem.mert hát szépen majd fognak feleltenve megyek a nagyon rövid úton.medve nos még akkor mikor.el hová menne az a medve..hol a.őb inkább még folytattaan a.- jelent ki egyszer édesap..csak ha beszélt hozzá.ne ne..hogy itt volt..ez szegény

In [ ]:
input_len = len(text.split())
k=0
while k<input_len/128:
    text_2 = text.split()[k*128:(k+1)*128] 
    text_2=" ".join(text_2)
    generated = context_gen_4(text_2, excluded_words)
    with open(genetaringfolderpath+'randomizalt_kimenet_context_15128parag.txt', 'a') as file_out:
        file_out.write(generated)
        file_out.write('.')
    k+=1
    with open(genetaringfolderpath+'csuszoablak_counter.txt', 'w') as file_out_k:
        file_out_k.write(str(k))
    print("progress :")
    print(k/(input_len/128))

progress :
0.09162491052254831
progress :
0.18324982104509663
progress :
0.27487473156764497
progress :
0.36649964209019326
progress :
0.4581245526127416
progress :
0.5497494631352899
progress :
0.6413743736578382
progress :
0.7329992841803865
progress :
0.8246241947029349
progress :
0.9162491052254832
progress :
1.0078740157480315
